In [206]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Dense, Input, Flatten, Dropout
from keras.optimizers import Adam, SGD
from keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy, BinaryCrossentropy
import os

dataset_path = "datasetMore/"
num_classes = 10
epochs = 500

In [93]:
def convert_img(img_path) -> np.ndarray:
    img = tf.keras.preprocessing.image.load_img(img_path, 
                                                grayscale=True, 
                                                color_mode="grayscale")
    output = tf.keras.preprocessing.image.img_to_array(img)
    output = tf.image.resize(output, size=(28,28))
    return output

In [85]:
def convert_label(label):
    output = tf.keras.utils.to_categorical(label, 
                                  num_classes)
    return output

In [162]:
def prepare_for_prediction(img_path):
    img = convert_img(img_path)
    img = tf.convert_to_tensor(img)
    
    img = tf.squeeze(img)
    
    return img

In [192]:
def load_data(dir_path): #-> tuple[tf.Tensor, tf.Tensor]:
    datas = []
    labels = []
    
    dirs = os.listdir(dir_path)
    for dir in dirs:
        
        file_path = os.path.join(dir_path, dir)
        for files_in_dir in os.listdir(file_path):
            
            img_array = convert_img(os.path.join(file_path, files_in_dir)) / 255
            datas.append(img_array)
            
            label = convert_label(dir)
            labels.append(label)

    
    return labels, datas

In [214]:
labels, datas = load_data(dataset_path)
X_train, X_test, y_train, y_test = train_test_split(labels, datas, test_size=0.4, shuffle=True, random_state=42)

X_train = tf.convert_to_tensor(X_train)
X_test = tf.convert_to_tensor(X_test)

y_train = tf.convert_to_tensor(y_train)
y_train = tf.squeeze(y_train)
y_test = tf.convert_to_tensor(y_test)
y_test = tf.squeeze(y_test)

inputLayer = Input(shape=(28,28))

#output = Dense(256, activation="relu", input_shape=(28,28))(inputLayer)
#output = Dropout(0.5)(output)
output = Dense(128, activation="relu")(inputLayer)
output = Dropout(0.5)(output)
output = Dense(68, activation="relu")(output)
output = Flatten()(output)
output = Dense(num_classes, activation="softmax")(output)

model = Model(inputLayer, output)
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=CategoricalCrossentropy(from_logits=True),
    metrics=["acc"]
)

model.summary()

c:\Users\Moritz\anaconda3\lib\site-packages\keras\utils\image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(


Model: "model_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_40 (InputLayer)       [(None, 28, 28)]          0         
                                                                 
 dense_123 (Dense)           (None, 28, 128)           3712      
                                                                 
 dropout_22 (Dropout)        (None, 28, 128)           0         
                                                                 
 dense_124 (Dense)           (None, 28, 68)            8772      
                                                                 
 flatten_16 (Flatten)        (None, 1904)              0         
                                                                 
 dense_125 (Dense)           (None, 10)                19050     
                                                                 
Total params: 31,534
Trainable params: 31,534
Non-trainabl

In [215]:
model.evaluate(y_test, X_test)
model.fit(y_train, X_train, validation_data=(y_test, X_test), epochs=epochs)
model.save('model.keras')

  1/270 [..............................] - ETA: 1:04 - loss: 2.3883 - acc: 0.0625

c:\Users\Moritz\anaconda3\lib\site-packages\keras\backend.py:5531: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


270/270 [==============================] - 1s 1ms/step - loss: 2.4145 - acc: 0.1082
Epoch 1/500
405/405 [==============================] - 3s 5ms/step - loss: 2.1719 - acc: 0.2135 - val_loss: 1.7903 - val_acc: 0.4291
Epoch 2/500
405/405 [==============================] - 2s 4ms/step - loss: 1.6858 - acc: 0.4615 - val_loss: 1.4487 - val_acc: 0.5515
Epoch 3/500
405/405 [==============================] - 2s 5ms/step - loss: 1.4301 - acc: 0.5558 - val_loss: 1.2923 - val_acc: 0.6050
Epoch 4/500
405/405 [==============================] - 2s 5ms/step - loss: 1.2580 - acc: 0.6145 - val_loss: 1.0815 - val_acc: 0.6758
Epoch 5/500
405/405 [==============================] - 2s 4ms/step - loss: 1.1438 - acc: 0.6490 - val_loss: 0.9966 - val_acc: 0.7034
Epoch 6/500
405/405 [==============================] - 2s 5ms/step - loss: 1.0533 - acc: 0.6780 - val_loss: 0.9320 - val_acc: 0.7222
Epoch 7/500
405/405 [==============================] - 2s 5ms/step - loss: 0.9880 - acc: 0.7014 - val_loss: 0.9084 - v

In [216]:
img = prepare_for_prediction("examples/Example_1.png")
input_value = tf.expand_dims(img, axis=0)

predict_value = model.predict(input_value)
print(predict_value)

1/1 [==============================] - 0s 76ms/step
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
